In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

## Code which scrapes the recently reviews (only 100 pages - 1,400 reviews)

In [2]:

def ratebeer_scraper(page_start=1, page_end=2, url_set=None):
    
    beer_name = []
    beer_rating = []
    brewery_name = []
    brewery_location = []
    review_content = []
    reviewer_name = []
 
    if url_set == None:
        print("Need url to run")
        return

    for x in tqdm(range(page_start, page_end + 1)):
        try:
            url_format = url_set.format(x)
            r = requests.get(url_format)
            soup = BeautifulSoup(r.text, 'html.parser')
            table = soup.find('table', attrs={'class': 'table'})
            table_rows = table.find_all('tr')

            for rows in table_rows:

            # Scrape Beer Name    
                try:
                    beer_name.append(rows.find('a', attrs={'style':'font-size:20px; font-weight:bold;'}).text)
                except:
                    beer_name.append(None)
            # Scrape Review Rating  
                try:
                    beer_rating.append(rows.find('span', attrs={'class':'uas'}).text)
                except:
                    beer_rating.append(None)
            # Scrape Brewery Name  
                try:
                    brewery_div = rows.find('div')
                    brewery_name.append(brewery_div.a.text)
                except:
                    brewery_name.append(None)
            # Scrape Brewery Location
                try:
                    brewery_location.append(rows.find('span', attrs={'class':'small'}).text.strip())
                except:
                    brewery_location.append(None)
            # Scrape Reviewer Name
                try:
                    reviewer_name.append(rows.find('span', attrs={'class':'pull-right'}).text.strip())
                except:
                    reviewer_name.append(None)
            # Scrape content
                try:
                    review_content.append(rows.find('div', attrs={'style':"color:#666;"}).text.strip())
                except:
                    review_content.append(None)
        except:
            pass

    beerz = pd.DataFrame({'beer_name': beer_name,
                         'beer_rating': beer_rating,
                        'brewery_name': brewery_name,
                         'brewery_location': brewery_location,
                        'review_content': review_content,
                        'reviewer_name': reviewer_name})
    return beerz



In [3]:
# Scraping most recent reviews
url = 'https://www.ratebeer.com/beer-ratings/0/{}/'
df_recent = ratebeer_scraper(0,100,url)

# Scraping new beers
url = 'https://www.ratebeer.com/beer-ratings/5/{}/'
df_new = ratebeer_scraper(0,10,url)

# Scraping fave beers
url = 'https://www.ratebeer.com/beer-ratings/4/{}/'
df_fave = ratebeer_scraper(0,50,url)

# Scraping rave beers
url = 'https://www.ratebeer.com/beer-ratings/1/{}/'
df_rave = ratebeer_scraper(0,50,url)

# Scraping rant beers
url = 'https://www.ratebeer.com/beer-ratings/2/{}/'
df_rant = ratebeer_scraper(0,50,url)

# Scraping anomalous beers
url = 'https://www.ratebeer.com/beer-ratings/3/{}/'
df_anom = ratebeer_scraper(0,50,url)

In [4]:
# getting rid of non-ascii characters from the reviews as it breaks csv

df_recent.review_content = df_recent.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_new.review_content = df_new.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_rave.review_content = df_rave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_fave.review_content = df_fave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_rant.review_content = df_rant.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_anom.review_content = df_anom.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))


In [5]:
# read current data from csv (there is a backup at all_reviews_backup.csv)
df = pd.read_csv('all_reviews.csv')

In [6]:
# concatenating all of the scrapes together with the old df
try:
    df = pd.concat([df, df_recent, df_new, df_fave, df_rave, df_rant, df_anom])
except:
    df = pd.concat([df, df_recent])
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
df.beer_name = df.beer_name.apply(lambda x: x.strip())



43857
39579


In [7]:
df

beer_name beer_rating  \
0                  St. Peter's Ruby Red Ale         3.3   
1          Shepherd Neame Spitfire (Bottle)         2.2   
2                           Arbor Rocketman         3.8   
3    Shepherd Neame India Pale Ale (Bottle)         3.4   
4              Shepherd Neame Christmas Ale         3.3   
..                                      ...         ...   
760        Solemn Oath Snaggletooth Bandana         2.0   
761   Cloudwater / Kees You've Been Spotted         1.8   
762                        Villages Whistle         1.5   
763           Wander Beyond Sakura Twilight         2.0   
764                   Smuttynose Vunderbar!         1.7   

             brewery_location                brewery_name  \
0             Bungay, Suffolk         St. Peter's Brewery   
1             Faversham, Kent              Shepherd Neame   
2    Bristol, Gloucestershire                  Arbor Ales   
3             Faversham, Kent              Shepherd Neame   
4             Faversham, Kent              Shepherd Neame   
..                        ...                         ...   
760                      -1.6         Solemn Oath Brewery   
761                      -1.7          Cloudwater Brew Co   
762                      -1.5            Villages Brewery   
763                      -1.7       Wander Beyond Brewing   
764                      -1.7  Smuttynose Brewing Company   

                                        review_content        reviewer_name  
0    Ruby color, offwhite coam. Aroma and taste: fl...            alex_leit  
1    Disappointing show from the lads at ‘Britain’s...  shivermetimber.gray  
2    Puszka. Mętne o żółtym kolorze. Piana biała,ob...              MarcinG  
3    Bottle. Amber with bronze color and white foam...            alex_leit  
4    Bottle. Aroma: English hops and malts with tof...            alex_leit  
..                                                 ...                  ...  
760  Pours a darker gold color. Decent white had at...              screigh  
761  [emailprotected]'Connell's, Tampere, Finland. ...           Turboninja  
762  330ml can share sample.  Murky golden with a t...                Beese  
763  Zeer donkerrood bier met beperkte beige schuim...      FrothyTickers24  
764  Pretty yuck. Although after having tons of bee...        BineChalice28  

[39579 rows x 6 columns]

In [8]:
df.to_csv('all_reviews.csv',index=False)
# df = pd.read_csv('all_reviews.csv')
not_found_df = pd.read_csv("not_found.csv")

## Get more information on all of the beers - Selenium

In [9]:
from selenium import webdriver
import time

In [10]:
# extract already scraped data from previous scrapes so don't rescrape
beer_df = pd.read_csv('beer_deets.csv')

In [11]:
# compare to the unique list to create a new to-be-scraped list
new_beer = df.beer_name[(~df.beer_name.isin(beer_df.name_found))&(~df.beer_name.isin(not_found_df.not_found))].unique()
print("There are", len(new_beer), "new beers to search for!")

There are 837 new beers to search for!


In [12]:
item_names = []
desc_string = []

driver = webdriver.Chrome(executable_path='/Users/lukebetham/Downloads/chromedriver1')

for beer in tqdm(new_beer):
    driver.get("https://www.ratebeer.com/search?beername={}".format(beer))
    time.sleep(2)
    item_tags = driver.find_elements_by_class_name('fg-1')

    x=1
    beer_temp = []
    for item in item_tags:        
        if x <2:
            try:
                item_names.append(item.find_element_by_class_name('fd-r').text)
                x+=1
            except:
                pass
            try:
                beer_temp.append(item.text)
            except:
                pass
    try:
        desc_string.append(beer_temp[4])
    except:
        desc_string.append(None)

driver.quit()

In [13]:
beer_deets = pd.DataFrame(desc_string, columns = ['return_string'])
beer_deets

return_string
0    Itchen Valley Wat Tyler\n🇬🇧 Old Ale • 5.0%\n20...
1    Cotleigh Long Eared Owl\n🇬🇧 Bitter - Premium /...
2    Marston's / Purkmistr Bohemian Schwarzbier\n🇬🇧...
3    Shepherd Neame / Port Brewing Grumpys Pale Ale...
4    Davenports Last Minute\n🇬🇧 Blonde Ale / Golden...
..                                                 ...
832  Muskoka Detour\n🇨🇦 ISA - Session IPA • 4.3%\n8...
833  Coors Slice Orange\n🇨🇦 Pale Lager - North Am. ...
834  Milwaukee Classic Lager\n🇺🇸 Pale Lager - North...
835  Lickinghole Creek Three Chopt Tripel (Bourbon ...
836  Nickel Brook Uber (Peach)\n🇨🇦 Berliner Weisse ...

[837 rows x 1 columns]

In [14]:
## cleaning the scraped data
beer_deets.dropna(inplace=True)
beer_deets = beer_deets[~beer_deets.return_string.str.startswith('Logo\n')].copy()
no_alcohol = beer_deets[~beer_deets.return_string.str.contains('%')].copy()
beer_deets = beer_deets[beer_deets.return_string.str.contains('%')].copy()


In [15]:
# sorting the scraped data into columns
beer_deets['name_found'] = beer_deets.return_string.apply(lambda x: x.split('\n')[0].strip())
beer_deets['flag'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
beer_deets['type'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
beer_deets['abv'] = beer_deets.return_string.apply(lambda x: float(x.split('\n')[1].split('•')[1].replace("%","")))
beer_deets['rb_overall_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[2])
beer_deets['rb_style_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[3])
beer_deets['rb_user_rating'] = beer_deets.return_string.apply(lambda x: x.split('\n')[4])
beer_deets['rb_rating_number'] = beer_deets.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))


In [16]:
# need to add in add check here for price


In [17]:
# sorting the no alcohol measure
no_alcohol['name_found'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[0].strip())
no_alcohol['flag'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
no_alcohol['type'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
no_alcohol['abv'] = no_alcohol.type.apply(lambda x: 0 if x[-1] == '0' else None)
no_alcohol['type'] = no_alcohol.type.apply(lambda x: x[:-1] if x[-1] == '0' else x)
no_alcohol['check'] = no_alcohol.return_string.apply(lambda x: len(x.split('\n')))
no_alcohol = no_alcohol[no_alcohol['check']>4].copy()
no_alcohol['rb_overall_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[2])
no_alcohol['rb_style_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[3])
no_alcohol['rb_user_rating'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[4])
no_alcohol['rb_rating_number'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))
no_alcohol.drop('check',inplace=True, axis=1)

no_alcohol

return_string  \
28   Avalanche The Goat's Pale Ale\n🇺🇸 Pale Ale - A...   
97   Twisted Acre Jalapeno IPA\n🇺🇸 IPA - Flavored\n...   
98   Twisted Acre Hazy DIPA\n🇺🇸 IIPA DIPA - Hazy / ...   
99   Twisted Acre Oktoberfest\n🇺🇸 Märzen / Marzen /...   
100  Twisted Acre Red Zone\n🇺🇸 IPA - Red\n-\n-\n3.0...   
103                    4J IPA\n🇺🇸 IPA\n-\n-\n2.80\n(1)   
184  Casey Fruit Stand Farmhouse Ale - Attika Cherr...   
263  Pabst Lonestar Rio Jade\n🇺🇸 Amber Lager - Intl...   
333  Third Space Why Wouldn't Cha?\n🇺🇸 IPA - Hazy /...   
364  River Bend Round the Bend Pale Ale\n🇺🇸 Pale Al...   
407  Crooked Stave Juicy Hazy IPA\n🇺🇸 IPA - Hazy / ...   
425  Teton Thai Dark Matters\n🇺🇸 Stout - Milk / Swe...   
431  Bellwoods Jelly King (Raspberry)\n🇨🇦 Sour / Wi...   
474  Virgil's Ale\n🇺🇸 Blonde Ale / Golden Ale0\n-\n...   
484  Maltstrom NEIPL 7\n🇨🇦 Pale Lager - India / Hop...   
485  Teton Thai Money Penny English Pale\n🇺🇸 Pale A...   
491  Teton Thai Session IPA\n🇺🇸 ISA - Session IPA\n...   
536  Burnt Tree Rye Not?\n🇺🇸 IPA - Rye\n-\n-\n3.35\...   
554  Sir John All Work and no Play Makes Serge a Du...   
605  Brasserie Générale BCBG\n🇨🇦 Pilsener - Czech /...   
648  Amchi Premium Hefe\n🇩🇪 Weissbier - Hefeweizen0...   
805  Corsaire Sky Drop\n🇨🇦 Sour / Wild Beer0\n-\n-\...   
834  Milwaukee Classic Lager\n🇺🇸 Pale Lager - North...   

                                            name_found flag  \
28                       Avalanche The Goat's Pale Ale   🇺🇸   
97                           Twisted Acre Jalapeno IPA   🇺🇸   
98                              Twisted Acre Hazy DIPA   🇺🇸   
99                            Twisted Acre Oktoberfest   🇺🇸   
100                              Twisted Acre Red Zone   🇺🇸   
103                                             4J IPA   🇺🇸   
184    Casey Fruit Stand Farmhouse Ale - Attika Cherry   🇺🇸   
263                            Pabst Lonestar Rio Jade   🇺🇸   
333                      Third Space Why Wouldn't Cha?   🇺🇸   
364                 River Bend Round the Bend Pale Ale   🇺🇸   
407                       Crooked Stave Juicy Hazy IPA   🇺🇸   
425                            Teton Thai Dark Matters   🇺🇸   
431                   Bellwoods Jelly King (Raspberry)   🇨🇦   
474                                       Virgil's Ale   🇺🇸   
484                                  Maltstrom NEIPL 7   🇨🇦   
485                Teton Thai Money Penny English Pale   🇺🇸   
491                             Teton Thai Session IPA   🇺🇸   
536                                Burnt Tree Rye Not?   🇺🇸   
554  Sir John All Work and no Play Makes Serge a Du...   🇨🇦   
605                            Brasserie Générale BCBG   🇨🇦   
648                                 Amchi Premium Hefe   🇩🇪   
805                                  Corsaire Sky Drop   🇨🇦   
834                            Milwaukee Classic Lager   🇺🇸   

                                       type  abv rb_overall_score  \
28                Pale Ale - American / APA  NaN                -   
97                           IPA - Flavored  NaN                -   
98          IIPA DIPA - Hazy / Double NEIPA  NaN                -   
99       Märzen / Marzen / Oktoberfest Bier  NaN                -   
100                               IPA - Red  NaN                -   
103                                     IPA  NaN                -   
184           Saison / Farmhouse - Flavored  0.0               93   
263             Amber Lager - Intl / Vienna  NaN                -   
333                      IPA - Hazy / NEIPA  NaN                -   
364               Pale Ale - American / APA  0.0                -   
407                      IPA - Hazy / NEIPA  NaN                -   
425                    Stout - Milk / Sweet  NaN                -   
431             Sour / Wild Beer - Flavored  0.0               99   
474                 Blonde Ale / Golden Ale  0.0                -   
484              Pale Lager - India / Hoppy  NaN                -   
485                      Pale A

In [18]:
beer_df = pd.concat([beer_df,beer_deets,no_alcohol])

In [19]:
beer_df

return_string flag  \
0    St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...   🇬🇧   
1    Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...   🇬🇧   
2    Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...   🇬🇧   
3    Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...   🇬🇧   
4    Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...   🇬🇧   
..                                                 ...  ...   
554  Sir John All Work and no Play Makes Serge a Du...   🇨🇦   
605  Brasserie Générale BCBG\n🇨🇦 Pilsener - Czech /...   🇨🇦   
648  Amchi Premium Hefe\n🇩🇪 Weissbier - Hefeweizen0...   🇩🇪   
805  Corsaire Sky Drop\n🇨🇦 Sour / Wild Beer0\n-\n-\...   🇨🇦   
834  Milwaukee Classic Lager\n🇺🇸 Pale Lager - North...   🇺🇸   

                                       type  abv rb_overall_score  \
0                 Bitter - Ordinary / Best   4.3               49   
1                 Bitter - Ordinary / Best   4.5               34   
2                                      IPA   6.0               95   
3                            IPA - English   6.1               46   
4                     Strong Ale - English   7.0               45   
..                                      ...  ...              ...   
554       IIPA DIPA - Imperial / Double IPA  NaN                -   
605               Pilsener - Czech / Svetlý  NaN                -   
648                  Weissbier - Hefeweizen  0.0                -   
805                        Sour / Wild Beer  0.0                -   
834   Pale Lager - North Am. / Light / Rice  0.0                1   

    rb_style_score rb_user_rating rb_rating_number  \
0               95           3.25              415   
1               59           3.05             1083   
2               97           3.74               59   
3               47           3.22              420   
4               46           3.18              409   
..             ...            ...              ...   
554              -           4.15                2   
605              -           3.53                3   
648              -           2.87                6   
805              -           4.32                5   
834             11           1.67               25   

                                            name_found  
0                             St. Peter's Ruby Red Ale  
1                     Shepherd Neame Spitfire (Bottle)  
2                                      Arbor Rocketman  
3               Shepherd Neame India Pale Ale (Bottle)  
4                         Shepherd Neame Christmas Ale  
..                                                 ...  
554  Sir John All Work and no Play Makes Serge a Du...  
605                            Brasserie Générale BCBG  
648                                 Amchi Premium Hefe  
805                                  Corsaire Sky Drop  
834                            Milwaukee Classic Lager  

[21243 rows x 9 columns]

In [20]:
# compare to the unique list to create a new to-be-scraped list
not_found_beer = df.beer_name[~df.beer_name.isin(beer_df.name_found)].unique()
print("There are", len(not_found_beer)-len(not_found_df), "new beers that were not found or returned errors. They will be added to the no search list.")
not_found_df = pd.concat([not_found_df,pd.DataFrame(not_found_beer,columns = ["not_found"])])
not_found_df.drop_duplicates(inplace=True)

There are 30 new beers that were not found or returned errors. They will be added to the no search list.


In [21]:
beer_df.to_csv('beer_deets.csv',index=False)
not_found_df.to_csv("not_found.csv", index=False)